Using this source for what nutrients are necessary:
https://www.hsph.harvard.edu/nutritionsource/vitamins/

and this source for all data:
https://fdc.nal.usda.gov/download-datasets.html

In [156]:
import pandas as pd
import os

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
path = os.getcwd()
data_path = path + '\\data'

print(data_path)

D:\Git_Repositories\min-max_nutrition\data


In [157]:
#List of all foundation foods
foods = pd.read_csv(data_path + '\\foundation_food.csv')

#The descriptions of every food in this particular database
desc = pd.read_csv(data_path + '\\food.csv')

foods = pd.merge(foods, desc, on='fdc_id')

food_cols_to_drop = ['NDB_number', 'footnote', 'data_type', 'food_category_id', 'publication_date']
foods.drop(food_cols_to_drop, axis=1, inplace=True)

#The amount of every nutrient for every food in the database
nutrient_match = pd.read_csv(data_path + '\\food_nutrient.csv')

#The names and units of every nutrient that's tracked
nutrient_names = pd.read_csv(data_path + '\\nutrient.csv')
nutrient_names.rename(columns={'id': 'nutrient_id'}, inplace=True)

nutrient_match = pd.merge(nutrient_match, nutrient_names, on='nutrient_id', how='left')

nutrient_match['nutrient_name'] = nutrient_match['name'] + ' (' + nutrient_match['unit_name'] + ')'

nutrient_cols_to_drop = ["id", 'nutrient_id', 'data_points', 'derivation_id', 'min', 'max', 'median', 'footnote', 'min_year_acqured', "nutrient_nbr", 'rank', 'name', 'unit_name']
nutrient_match.drop(nutrient_cols_to_drop, axis=1, inplace=True)

nutrient_match.head()


C:\Users\chris\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,fdc_id,amount,nutrient_name
0,319877,56.30,Water (G)
1,319877,1.28,Nitrogen (G)
2,319877,19.00,Total lipid (fat) (G)
3,319877,1.98,Ash (G)
4,319878,188.00,"Phosphorus, P (MG)"


In [158]:
df1 = foods
df2 = nutrient_match


# Merge df1 and df2 based on 'fdc_id'
merged_df = pd.merge(df1, df2, on='fdc_id', how='right')

# Pivot the merged DataFrame to create columns for unique nutrient_names
pivoted_df = merged_df.pivot(index='fdc_id', columns='nutrient_name', values='amount')

# Merge the pivoted DataFrame back to df1 based on 'fdc_id'
pivoted_df = pd.merge(df1, pivoted_df, on='fdc_id')


pivoted_df.head()



,fdc_id,description,NaN,"Ergosta-5,7-dienol (MG)","Ergosta-7,22-dienol (MG)",10-Formyl folic acid (10HCOFA) (UG),25-hydroxycholecalciferol (UG),5-Formyltetrahydrofolic acid (5-HCOH4 (UG),5-methyl tetrahydrofolate (5-MTHF) (UG),Alanine (G),Arginine (G),Ash (G),Aspartic acid (G),Beta-glucan (G),Beta-sitostanol (MG),Beta-sitosterol (MG),Betaine (MG),Biotin (UG),"Boron, B (UG)",Brassicasterol (MG),"Calcium, Ca (MG)",Campestanol (MG),Campesterol (MG),"Carbohydrate, by difference (G)","Carbohydrate, by summation (G)","Carotene, alpha (UG)","Carotene, beta (UG)","Carotene, gamma (UG)",Cholesterol (MG),"Choline, free (MG)","Choline, from glycerophosphocholine (MG)","Choline, from phosphocholine (MG)","Choline, from phosphotidyl choline (MG)","Choline, from sphingomyelin (MG)","Choline, total (MG)",Citric acid (MG),"Cobalt, Co (UG)","Copper, Cu (MG)","Cryptoxanthin, alpha (UG)","Cryptoxanthin, beta (UG)",Cysteine (G),Cystine (G),Daidzein (MG),Daidzin (MG),Delta-5-avenasterol (MG),Delta-7-Stigmastenol (MG),Energy (Atwater General Factors) (KCAL),Energy (Atwater Specific Factors) (KCAL),Energy (KCAL),Energy (kJ),Ergosta-7-enol (MG),Ergosterol (MG),Ergothioneine (MG),"Fatty acids, total monounsaturated (G)","Fatty acids, total polyunsaturated (G)","Fatty acids, total saturated (G)","Fatty acids, total trans (G)","Fatty acids, total trans-dienoic (G)","Fatty acids, total trans-monoenoic (G)","Fatty acids, total trans-polyenoic (G)","Fiber, insoluble (G)","Fiber, soluble (G)","Fiber, total dietary (G)","Folate, total (UG)",Fructose (G),Galactose (G),Genistein (MG),Genistin (MG),Glucose (G),Glutamic acid (G),Glycine (G),Glycitin (MG),High Molecular Weight Dietary Fiber (HMWDF) (G),Histidine (G),Hydroxyproline (G),"Iodine, I (UG)","Iron, Fe (MG)",Isoleucine (G),Lactose (G),Leucine (G),Low Molecular Weight Dietary Fiber (LMWDF) (G),Lutein (UG),Lutein + zeaxanthin (UG),Lycopene (UG),Lysine (G),MUFA 12:1 (G),MUFA 14:1 c (G),MUFA 15:1 (G),MUFA 16:1 c (G),MUFA 17:1 (G),MUFA 17:1 c (G),MUFA 18:1 (G),MUFA 18:1 c (G),MUFA 20:1 (G),MUFA 20:1 c (G),MUFA 22:1 (G),MUFA 22:1 c (G),MUFA 22:1 n-11 (G),MUFA 22:1 n-9 (G),MUFA 24:1 c (G),"Magnesium, Mg (MG)",Malic acid (MG),Maltose (G),"Manganese, Mn (MG)",Methionine (G),"Molybdenum, Mo (UG)",Niacin (MG),"Nickel, Ni (UG)",Nitrogen (G),Oxalic acid (MG),PUFA 18:2 (G),PUFA 18:2 CLAs (G),PUFA 18:2 c (G),"PUFA 18:2 n-6 c,c (G)",PUFA 18:3 (G),PUFA 18:3 c (G),"PUFA 18:3 n-3 c,c,c (ALA) (G)","PUFA 18:3 n-6 c,c,c (G)",PUFA 18:3i (G),PUFA 18:4 (G),PUFA 20:2 c (G),"PUFA 20:2 n-6 c,c (G)",PUFA 20:3 (G),PUFA 20:3 c (G),PUFA 20:3 n-3 (G),PUFA 20:3 n-9 (G),PUFA 20:4 (G),PUFA 20:4 n-6 (G),PUFA 20:4c (G),PUFA 20:5 n-3 (EPA) (G),PUFA 20:5c (G),PUFA 22:2 (G),PUFA 22:3 (G),PUFA 22:4 (G),PUFA 22:5 c (G),PUFA 22:5 n-3 (DPA) (G),PUFA 22:6 c (G),PUFA 22:6 n-3 (DHA) (G),Pantothenic acid (MG),Phenylalanine (G),"Phosphorus, P (MG)",Phytoene (UG),Phytofluene (UG),"Phytosterols, other (MG)","Potassium, K (MG)",Proline (G),Protein (G),Pyruvic acid (MG),Quinic acid (MG),Raffinose (G),Retinol (UG),Riboflavin (MG),SFA 10:0 (G),SFA 11:0 (G),SFA 12:0 (G),SFA 14:0 (G),SFA 15:0 (G),SFA 16:0 (G),SFA 17:0 (G),SFA 18:0 (G),SFA 20:0 (G),SFA 21:0 (G),SFA 22:0 (G),SFA 23:0 (G),SFA 24:0 (G),SFA 4:0 (G),SFA 5:0 (G),SFA 6:0 (G),SFA 7:0 (G),SFA 8:0 (G),SFA 9:0 (G),"Selenium, Se (UG)",Serine (G),"Sodium, Na (MG)",Specific Gravity (SP_GR),Stachyose (G),Starch (G),Stigmastadiene (MG),Stigmasterol (MG),Sucrose (G),"Sugars, Total (G)","Sugars, total including NLEA (G)","Sulfur, S (MG)",TFA 14:1 t (G),TFA 16:1 t (G),TFA 18:1 t (G),TFA 18:2 t (G),TFA 18:2 t not further defined (G),TFA 18:3 t (G),TFA 20:1 t (G),TFA 22:1 t (G),Thiamin (MG),Threonine (G),"Tocopherol, beta (MG)","Tocopherol, delta (MG)","Tocopherol, gamma (MG)","Tocotrienol, alpha (MG)","Tocotrienol, beta (MG)","Tocotrienol, delta (MG)","Tocotrienol, gamma (MG)",Total dietary fiber (AOAC 2011.25) (G),Total fat (NLEA) (G),Total lipid (fat) (G),Tryptophan (G),Tyrosine (G),Valine (G),Verbascose (G),"Vit

Now we basically have all the data we need, but there are some rows that don't have explicitly-stated calorie information, so we need to calculate those things based on other data. For example, some rows don't have values for "Total Fat Content", so we need to calculate that based on values in other columns, or infer based on what the food actually is.


In [159]:
#First change all the rows without Total Fat.
#Most of these are cooking oils (except salt), so I will just put 100 for those.
pivoted_df.loc[pivoted_df['description'] == 'Salt, table, iodized', 'Total lipid (fat) (G)'] = 0
rows_with_nan = pivoted_df[pivoted_df["Total lipid (fat) (G)"].isna()]
pivoted_df.loc[rows_with_nan.index, "Total lipid (fat) (G)"] = 100


#Next, Protein
rows_with_nan = pivoted_df[pivoted_df["Protein (G)"].isna()]
pivoted_df.loc[rows_with_nan.index, "Protein (G)"] = 0

#Next, carbs
oils = ['Oil, canola', 'Oil, corn', 'Oil, soybean', 'Oil, olive, extra virgin', 'Butter, stick, unsalted', 'Butter, stick, salted', 'Oil, peanut', 'Oil, sunflower', 'Oil, safflower', 'Oil, olive, extra light']
pivoted_df.loc[pivoted_df['description'].isin(oils), 'Carbohydrate, by difference (G)'] = 0
pivoted_df.loc[pivoted_df['description'] == 'Salt, table, iodized', 'Carbohydrate, by difference (G)'] = 0

mask = pivoted_df['Carbohydrate, by difference (G)'].isnull()
sum_values = pivoted_df.loc[mask, 'Fiber, total dietary (G)'].add(pivoted_df.loc[mask, 'Starch (G)'], fill_value=0)
pivoted_df.loc[mask, 'Carbohydrate, by difference (G)'] = sum_values

#Now calculate Calories for all rows where this is missing

mask = pivoted_df['Energy (KCAL)'].isnull()
new_values = 9 * pivoted_df.loc[mask, 'Total lipid (fat) (G)'] + 4 * pivoted_df.loc[mask, 'Protein (G)'] + 4 * pivoted_df.loc[mask, 'Carbohydrate, by difference (G)']
pivoted_df.loc[mask, 'Energy (KCAL)'] = new_values

In [160]:
nutrients_dict = {
    'Biotin (UG)' : 30,
    'Calcium, Ca (MG)' : 1000,
    'Chromium, Cr (UG)' : 35,
    'Choline, total (MG)' : 550,
    'Copper, Cu (MG)' : 0.9,
    'Folate, total (UG)' : 400,
    'Iodine, I (UG)' : 150,
    'Iron, Fe (MG)' : 8,
    'Magnesium, Mg (MG)' : 400,
    'Manganese, Mn (MG)' : 2.3,
    'Molybdenum, Mo (UG)' : 45,
    'Niacin (MG)' : 16,
    'Nickel, Ni (UG)' : 35,
    'Pantothenic acid (MG)' : 5,
    'Phosphorus, P (MG)' : 700,
    'Potassium, K (MG)' : 3400,
    'Riboflavin (MG)' : 1.3,
    'Selenium, Se (UG)' : 55,
    'Sodium, Na (MG)' : 1500,
    'Thiamin (MG)' : 1.2,
    'Vitamin A, RAE (UG)' : 900,
    'Vitamin B-12 (UG)' : 2.4,
    'Vitamin B-6 (MG)' : 1.3,
    'Vitamin C, total ascorbic acid (MG)' : 90,
    'Vitamin D (D2 + D3) (UG)' : 15,
    'Vitamin E (alpha-tocopherol) (MG)' : 15,
    'Vitamin K (phylloquinone) (UG)' : 120,
    'Zinc, Zn (MG)' : 11,
}

keep_columns = list(nutrients_dict.keys())

columns_to_drop = [col for col in final_df.columns if col not in keep_columns]
columns_to_drop.append('description')
final_df = pivoted_df.drop(columns_to_drop, axis=1)
final_df.fillna(0, inplace=True)
final_df.head()

,Biotin (UG),"Calcium, Ca (MG)","Choline, total (MG)","Copper, Cu (MG)","Folate, total (UG)","Iodine, I (UG)","Iron, Fe (MG)","Magnesium, Mg (MG)","Manganese, Mn (MG)","Molybdenum, Mo (UG)",Niacin (MG),"Nickel, Ni (UG)",Pantothenic acid (MG),"Phosphorus, P (MG)","Potassium, K (MG)",Riboflavin (MG),"Selenium, Se (UG)","Sodium, Na (MG)",Thiamin (MG),"Vitamin A, RAE (UG)",Vitamin B-12 (UG),Vitamin B-6 (MG),"Vitamin C, total ascorbic acid (MG)",Vitamin D (D2 + D3) (UG),Vitamin E (alpha-tocopherol) (MG),Vitamin K (phylloquinone) (UG),"Zinc, Zn (MG)"
0,0.0,41.0,46.6,0.348,36.0,0.0,2.41,71.1,1.060,0.0,0.948,0.0,0.318,166.0,289.0,0.115,16.2,438.0,0.150,1.0,0.00,0.143,0.0,0.0,1.74,17.2,1.38
1,0.0,11.0,9.8,0.058,10.0,0.0,0.33,11.9,0.121,0.0,0.805,0.0,0.000,28.0,260.0,0.065,0.0,6.0,0.075,0.0,0.00,0.060,27.2,0.0,0.98,4.2,0.20
2,0.0,36.0,0.0,0.041,0.0,0.0,0.78,12.7,0.176,0.0,0.000,0.0,0.000,23.0,97.0,0.000,0.0,282.0,0.000,0.0,0.00,0.000,0.0,0.0,0.00,0.0,0.19
3,0.0,15.0,0.0,0.046,0.0,0.0,1.14,11.5,0.031,0.0,2.250,0.0,0.263,128.0,343.0,0.154,0.0,872.0,0.033,3.0,0.97,0.130,0.0,0.0,0.51,0.0,2.06
4,0.0,273.0,60.8,0.870,35.0,0.0,3.17,258.0,2.020,0.0,3.100,0.0,0.237,456.0,684.0,1.570,0.0,256.0,0.079,2.0,0.00,0.075,0.0,0.0,19.00,0.0,2.80


In [154]:
# Assume 'df' is your DataFrame

# Count the number of NaN values in each column
nan_counts = pivoted_df.isna().sum()


print(nan_counts.sort_values(ascending=True))

fdc_id                                               0
description                                          0
Energy (KCAL)                                        0
Total lipid (fat) (G)                                0
Carbohydrate, by difference (G)                      0
Protein (G)                                          0
Water (G)                                            8
Zinc, Zn (MG)                                       12
Iron, Fe (MG)                                       12
Magnesium, Mg (MG)                                  12
Manganese, Mn (MG)                                  12
Calcium, Ca (MG)                                    12
Copper, Cu (MG)                                     12
Phosphorus, P (MG)                                  12
Potassium, K (MG)                                   12
Ash (G)                                             27
Energy (Atwater General Factors) (KCAL)             28
Sodium, Na (MG)                                     30
Nitrogen (

In [152]:


# Find the row(s) where the value in the "Ash (G)" column is NaN
rows_with_nan = pivoted_df[pivoted_df["Energy (KCAL)"].isna()]

#rows_with_nan.drop(columns=keep_columns, inplace=True)

# Print the row(s) with NaN value in the "Ash (G)" column
rows_with_nan.dropna(axis=1, how='all', inplace=True)

rows_with_nan

C:\Users\chris\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


,fdc_id,description,"Ergosta-5,7-dienol (MG)","Ergosta-7,22-dienol (MG)",25-hydroxycholecalciferol (UG),Alanine (G),Arginine (G),Ash (G),Aspartic acid (G),Beta-glucan (G),Beta-sitostanol (MG),Beta-sitosterol (MG),Biotin (UG),"Boron, B (UG)",Brassicasterol (MG),"Calcium, Ca (MG)",Campestanol (MG),Campesterol (MG),"Carbohydrate, by difference (G)","Carbohydrate, by summation (G)","Carotene, alpha (UG)","Carotene, beta (UG)","Carotene, gamma (UG)",Cholesterol (MG),Citric acid (MG),"Cobalt, Co (UG)","Copper, Cu (MG)","Cryptoxanthin, alpha (UG)","Cryptoxanthin, beta (UG)",Cysteine (G),Daidzein (MG),Daidzin (MG),Delta-5-avenasterol (MG),Delta-7-Stigmastenol (MG),Energy (Atwater General Factors) (KCAL),Energy (Atwater Specific Factors) (KCAL),Ergosta-7-enol (MG),Ergosterol (MG),Ergothioneine (MG),"Fatty acids, total monounsaturated (G)","Fatty acids, total polyunsaturated (G)","Fatty acids, total saturated (G)","Fatty acids, total trans (G)","Fatty acids, total trans-monoenoic (G)","Fatty acids, total trans-polyenoic (G)","Fiber, total dietary (G)","Folate, total (UG)",Fructose (G),Galactose (G),Genistein (MG),Genistin (MG),Glucose (G),Glutamic acid (G),Glycine (G),Glycitin (MG),High Molecular Weight Dietary Fiber (HMWDF) (G),Histidine (G),Hydroxyproline (G),"Iodine, I (UG)","Iron, Fe (MG)",Isoleucine (G),Lactose (G),Leucine (G),Low Molecular Weight Dietary Fiber (LMWDF) (G),Lutein (UG),Lutein + zeaxanthin (UG),Lycopene (UG),Lysine (G),MUFA 12:1 (G),MUFA 14:1 c (G),MUFA 15:1 (G),MUFA 16:1 c (G),MUFA 17:1 (G),MUFA 17:1 c (G),MUFA 18:1 c (G),MUFA 20:1 c (G),MUFA 22:1 c (G),MUFA 22:1 n-11 (G),MUFA 22:1 n-9 (G),MUFA 24:1 c (G),"Magnesium, Mg (MG)",Malic acid (MG),Maltose (G),"Manganese, Mn (MG)",Methionine (G),"Molybdenum, Mo (UG)",Niacin (MG),"Nickel, Ni (UG)",Nitrogen (G),Oxalic acid (MG),PUFA 18:2 CLAs (G),PUFA 18:2 c (G),"PUFA 18:2 n-6 c,c (G)",PUFA 18:3 c (G),"PUFA 18:3 n-3 c,c,c (ALA) (G)","PUFA 18:3 n-6 c,c,c (G)",PUFA 18:4 (G),PUFA 20:2 c (G),"PUFA 20:2 n-6 c,c (G)",PUFA 20:3 c (G),PUFA 20:3 n-3 (G),PUFA 20:3 n-9 (G),PUFA 20:4 (G),PUFA 20:4 n-6 (G),PUFA 20:4c (G),PUFA 20:5 n-3 (EPA) (G),PUFA 20:5c (G),PUFA 22:2 (G),PUFA 22:3 (G),PUFA 22:4 (G),PUFA 22:5 c (G),PUFA 22:5 n-3 (DPA) (G),PUFA 22:6 c (G),PUFA 22:6 n-3 (DHA) (G),Phenylalanine (G),"Phosphorus, P (MG)","Phytosterols, other (MG)","Potassium, K (MG)",Proline (G),Protein (G),Pyruvic acid (MG),Quinic acid (MG),Raffinose (G),Retinol (UG),Riboflavin (MG),SFA 10:0 (G),SFA 11:0 (G),SFA 12:0 (G),SFA 14:0 (G),SFA 15:0 (G),SFA 16:0 (G),SFA 17:0 (G),SFA 18:0 (G),SFA 20:0 (G),SFA 21:0 (G),SFA 22:0 (G),SFA 23:0 (G),SFA 24:0 (G),SFA 4:0 (G),SFA 5:0 (G),SFA 6:0 (G),SFA 7:0 (G),SFA 8:0 (G),SFA 9:0 (G),"Selenium, Se (UG)",Serine (G),"Sodium, Na (MG)",Stachyose (G),Starch (G),Stigmastadiene (MG),Stigmasterol (MG),Sucrose (G),"Sugars, Total (G)","Sugars, total including NLEA (G)","Sulfur, S (MG)",TFA 14:1 t (G),TFA 16:1 t (G),TFA 18:1 t (G),TFA 18:2 t (G),TFA 18:2 t not further defined (G),TFA 18:3 t (G),TFA 20:1 t (G),TFA 22:1 t (G),Thiamin (MG),Threonine (G),"Tocopherol, beta (MG)","Tocopherol, delta (MG)","Tocopherol, gamma (MG)","Tocotrienol, alpha (MG)","Tocotrienol, beta (MG)","Tocotrienol, delta (MG)","Tocotrienol, gamma (MG)",Total dietary fiber (AOAC 2011.25) (G),Total fat (NLEA) (G),Total lipid (fat) (G),Tryptophan (G),Tyrosine (G),Valine (G),Verbascose (G),"Vitamin A, RAE (UG)",Vitamin B-12 (UG),Vitamin B-6 (MG),"Vitamin C, total ascorbic acid (MG)",Vitamin D (D2 + D3) (UG),"Vitamin D (D2 + D3), International Units (IU)",Vitamin D2 (ergocalciferol) (UG),Vitamin D3 (cholecalciferol) (UG),Vitamin D4 (UG),Vitamin E (alpha-tocopherol) (MG),Vitamin K (Dihydrophylloquinone) (UG),Vitamin K (Menaquinone-4) (UG),Vitamin K (phylloquinone) (UG),Water (G),Zeaxanthin (UG),"Zinc, Zn (MG)",cis-Lutein/Zeaxanthin (UG),cis-Lycopene (UG),cis-beta-Carotene (UG),trans-Lycopene (UG),trans-beta-Carotene (UG)
61,746775,"Salt, table, iodized",NaN,NaN,NaN,NaN,NaN,99.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50.0000

In [ ]:
[
    ''
]

In [104]:
temp = pd.read_csv(data_path + '\\nutrient.csv')

temp

,id,name,unit_name,nutrient_nbr,rank
0,2047,Energy (Atwater General Factors),KCAL,957.00,280.0
1,2048,Energy (Atwater Specific Factors),KCAL,958.00,290.0
2,1001,Solids,G,201.00,200.0
3,1002,Nitrogen,G,202.00,500.0
4,1003,Protein,G,203.00,600.0
5,1004,Total lipid (fat),G,204.00,800.0
6,1005,"Carbohydrate, by difference",G,205.00,1110.0
7,1006,"Fiber, crude (DO NOT USE - Archived)",G,206.00,999999.0
8,1007,Ash,G,207.00,1000.0
9,1008,Energy,KCAL,208.00,300.0
